# Preprocess text

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_11a import *

## Loading and splitting the dataset

In [3]:
path = datasets.untar_data(datasets.URLs.IMDB)

In [4]:
path.ls()

[PosixPath('/home/fabiograetz/.fastai/data/imdb/README'),
 PosixPath('/home/fabiograetz/.fastai/data/imdb/train'),
 PosixPath('/home/fabiograetz/.fastai/data/imdb/test'),
 PosixPath('/home/fabiograetz/.fastai/data/imdb/ll_clas.pkl'),
 PosixPath('/home/fabiograetz/.fastai/data/imdb/unsup'),
 PosixPath('/home/fabiograetz/.fastai/data/imdb/tmp_clas'),
 PosixPath('/home/fabiograetz/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/home/fabiograetz/.fastai/data/imdb/tmp_lm')]

In [5]:
#export
def read_file(fn): 
    with open(fn, 'r', encoding = 'utf8') as f: return f.read()

In [6]:
#export
class TextList(ItemList):
    @classmethod
    def from_files(cls, path, extensions='.txt', recurse=True, include=None, **kwargs):
        return cls(get_files(path, extensions, recurse=recurse, include=include), path, **kwargs)
    
    def get(self, i):
        if isinstance(i, Path): return read_file(i)
        return i

In [7]:
il = TextList.from_files(path, include=['train', 'test', 'unsup'])

In [8]:
len(il)

100000

In [9]:
il

TextList (100000 items)
 [PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/9809_2.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/7291_2.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/1279_3.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/7323_1.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/9921_3.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/1825_2.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/233_1.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/3324_3.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/9439_3.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/10967_4.txt')...]
 Path: /home/fabiograetz/.fastai/data/imdb

In [10]:
text = il[0]

In [11]:
text

"Some wonder why there weren't anymore Mrs. Murphy movies after this one. Will it's because this movie totally blew snot. Disney was not the right studio to run this film. MAYBE Touchstone (well, they're owned by Disney, but it'd be more adult). The film is too kid-ish, as the book series is not. The casting is all wrong for the characters. The characters don't even act the way they do in the books. And why was Tucker changed to a guy? He's a girl in the frigging books! Was this done to make the film appeal to boys? Sheesh. And where was Pewter, the gray cat? One of the funniest characters from the book is absent from this filth. Rita Mae Brown is a good writer, but letting Disney blow her work was wrong. An animated feature film, perhaps in the vane of Don Bluth's artwork would suit a better Mrs. Murphy film. Overall, I give this a 2, because at least Disney made a film from an under-appreciated book series. But, I wish they did better. Either way, I still have my books to entertain m

In [12]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [13]:
sd

SplitData
Train: TextList (89874 items)
 [PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/9809_2.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/7291_2.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/1279_3.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/7323_1.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/9921_3.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/1825_2.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/233_1.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/3324_3.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/9439_3.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/10967_4.txt')...]
 Path: /home/fabiograetz/.fastai/data/imdb
Valid: TextList (10126 items)
 [PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/3059_1.txt'), PosixPath('/home/fabiograetz/.fastai/data/imdb/train/neg/845_1.txt'), PosixPath('/home/fabiograetz/

## Tokenization

In [14]:
#export
import spacy, html

In [15]:
#export
UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxunk xxpad xxbos xxeos xxrep xxwrep xxup xxmaj".split()

def sub_br(t):
    "Replaces the <br /> by \n"
    re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
    return re_br.sub("\n", t)

def spec_add_spaces(t):
    "Add spaces around / and #"
    return re.sub(r'([/#])', r' \1 ', t)

def rm_useless_spaces(t):
    "Remove multiple spaces"
    return re.sub(' {2,}', ' ', t)

def replace_rep(t):
    "Replace repetitions at the character level: cccc -> TK_REP 4 c"
    def _replace_rep(m:Collection[str]) -> str:
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    re_rep = re.compile(r'(\S)(\1{3,})')
    return re_rep.sub(_replace_rep, t)
    
def replace_wrep(t):
    "Replace word repetitions: word word word -> TK_WREP 3 word"
    def _replace_wrep(m:Collection[str]) -> str:
        c,cc = m.groups()
        return f' {TK_WREP} {len(cc.split())+1} {c} '
    re_wrep = re.compile(r'(\b\w+\W+)(\1{3,})')
    return re_wrep.sub(_replace_wrep, t)

def fixup_text(x):
    "Various messy things we've seen in documents"
    re1 = re.compile(r'  +')
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

In [16]:
#export
default_pre_rules = [fixup_text, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces, sub_br]

default_spec_tok = [UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ]

In [17]:
replace_rep('aaaa')

' xxrep 4 a '

In [18]:
replace_wrep("test test test test ")

' xxwrep 4 test  '

*Rules that are applied after tokenization:*

In [19]:
#export
def replace_all_caps(x):
    "Replace tokens in ALL CAPS by their lower version and add `TK_UP` before."
    res = []
    for t in x:
        if t.isupper() and len(t) > 1: res.append(TK_UP); res.append(t.lower())
        else: res.append(t)
    return res

In [20]:
replace_all_caps(["AAA", "bbb", "Fabio" ,"FABIO"])

['xxup', 'aaa', 'bbb', 'Fabio', 'xxup', 'fabio']

In [21]:
#export
def deal_caps(x):
    "Replace all Capitalized tokens by their lower version and add `TK_MAJ` before."
    res = []
    for t in x:
        if t == '': continue
        if t[0].isupper() and len(t) > 1 and t[1:].islower(): 
            res.append(TK_MAJ)
        res.append(t.lower())
    return res

In [22]:
deal_caps(["AAA", "bbb", "Fabio" ,"FABIO"])

['aaa', 'bbb', 'xxmaj', 'fabio', 'fabio']

In [23]:
#export
def add_eos_bos(x): return [BOS] + x + [EOS]

In [24]:
#export
default_post_rules = [replace_all_caps, deal_caps, add_eos_bos]  # changed order with respect to fastai because otherwise all_caps are not handled correctly

In [25]:
x = ["AAA", "bbb", "Fabio" ,"FABIO"]

for f in default_post_rules:
    x = f(x)

In [26]:
x

['xxbos', 'xxup', 'aaa', 'bbb', 'xxmaj', 'fabio', 'xxup', 'fabio', 'xxeos']

In [27]:
#export
from spacy.symbols import ORTH
from concurrent.futures import ProcessPoolExecutor

In [28]:
#export
def parallel(func, arr, max_workers=4):
    if max_workers < 2:
        results = list(progress_bar(map(func, enumerate(arr)), total=len(arr)))
    else:
        with ProcessPoolExecutor(max_workers=max_workers) as ex:
            return list(progress_bar(ex.map(func, enumerate(arr)), total=len(arr)))
    if any([o is not None for o in results]): return results

In [29]:
#export
class TokenizeProcessor(Processor):
    def __init__(self, lang="en", chunksize=2000, pre_rules=None, post_rules=None, max_workers=4): 
        self.chunksize, self.max_workers = chunksize, max_workers
        self.tokenizer = spacy.blank(lang).tokenizer
        
        for w in default_spec_tok:
            self.tokenizer.add_special_case(w, [{ORTH: w}])
        
        self.pre_rules  = default_pre_rules  if pre_rules  is None else pre_rules
        self.post_rules = default_post_rules if post_rules is None else post_rules

    def proc_chunk(self, args):
        # chunk is a list of strings
        i, chunk = args
        chunk = [compose(t, self.pre_rules) for t in chunk]  # list of strings
        docs = [[d.text for d in doc] for doc in self.tokenizer.pipe(chunk)]  # docs is a list of lists of tokens
        docs = [compose(t, self.post_rules) for t in docs]  # Formerly capitalized tokens are all lowercase now with special tokens before
        return docs  # List of lists of tokens
 
    def __call__(self, items): 
        toks = []
        if isinstance(items[0], Path): items = [read_file(i) for i in items]
        # items is a list of strings
        chunks = [items[i: i+self.chunksize] for i in (range(0, len(items), self.chunksize))]
        # chunks is a list of lists of strings

        toks = parallel(self.proc_chunk, chunks, max_workers=self.max_workers)
        return sum(toks, [])
    
    def proc1(self, item): return self.proc_chunk([item])[0]
    
    def deprocess(self, toks): return [self.deproc1(tok) for tok in toks]
    def deproc1(self, tok):    return " ".join(tok)

In [30]:
tp = TokenizeProcessor()

In [31]:
text[:200]

"Some wonder why there weren't anymore Mrs. Murphy movies after this one. Will it's because this movie totally blew snot. Disney was not the right studio to run this film. MAYBE Touchstone (well, they'"

In [32]:
' • '.join(tp(il[:10])[0])[:400]

"xxbos • xxmaj • some • wonder • why • there • were • n't • anymore • xxmaj • mrs. • xxmaj • murphy • movies • after • this • one • . • xxmaj • will • it • 's • because • this • movie • totally • blew • snot • . • xxmaj • disney • was • not • the • right • studio • to • run • this • film • . • xxup • maybe • xxmaj • touchstone • ( • well • , • they • 're • owned • by • xxmaj • disney • , • but • it"

## Numericalization

In [33]:
#export
import collections

class NumericalizeProcessor(Processor):
    def __init__(self, vocab=None, max_vocab=60000, min_freq=2):
        self.vocab, self.max_vocab, self.min_freq = vocab, max_vocab, min_freq
        
    def __call__(self, items):
        # items is a list of lists of tokens
        # Define vocab on first use
        if self.vocab is None:
            freq = Counter(p for o in items for p in o)
            self.vocab = [o for o, c in freq.most_common(self.max_vocab) if c >= self.min_freq]
            
            for o in reversed(default_spec_tok):
                if o in self.vocab: self.vocab.remove(o)
                self.vocab.insert(0, o)
                
        if getattr(self, 'otoi', None) is None:
            self.otoi = collections.defaultdict(int, {v:k for k,v in enumerate(self.vocab)})
        
        return [self.proc1(o) for o in items]
    
    def proc1(self, item):
        # item is list of tokens
        return [self.otoi[o] for o in item]  # returns list of ints
    
    def deprocess(self, idxs):
        #idxs is a list of lists of ints
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    
    def deproc1(self, idx):
        # idx is a list of ints
        return [self.vocab[i] for i in idx]

In [34]:
proc_tok, proc_num = TokenizeProcessor(max_workers=8), NumericalizeProcessor()

In [35]:
%time ll = label_by_func(sd, lambda x: 0, proc_x= [proc_tok, proc_num])

CPU times: user 17.5 s, sys: 1.92 s, total: 19.4 s
Wall time: 42 s


In [36]:
idxs = proc_num.proc1(["xxbos", "xxmaj", "some", "wonder", "why", "there", "were", "n't", "anymore"])

In [37]:
idxs

[2, 7, 65, 606, 154, 54, 86, 35, 1561]

In [38]:
proc_num.deproc1(idxs)

['xxbos', 'xxmaj', 'some', 'wonder', 'why', 'there', 'were', "n't", 'anymore']

In [39]:
print(ll.train.x_obj(0))

xxbos xxmaj some wonder why there were n't anymore xxmaj mrs. xxmaj murphy movies after this one . xxmaj will it 's because this movie totally blew snot . xxmaj disney was not the right studio to run this film . xxup maybe xxmaj touchstone ( well , they 're owned by xxmaj disney , but it 'd be more adult ) . xxmaj the film is too kid - ish , as the book series is not . xxmaj the casting is all wrong for the characters . xxmaj the characters do n't even act the way they do in the books . xxmaj and why was xxmaj tucker changed to a guy ? xxmaj he 's a girl in the frigging books ! xxmaj was this done to make the film appeal to boys ? xxmaj sheesh . xxmaj and where was xxmaj pewter , the gray cat ? xxmaj one of the funniest characters from the book is absent from this filth . xxmaj rita xxmaj mae xxmaj brown is a good writer , but letting xxmaj disney blow her work was wrong . xxmaj an animated feature film , perhaps in the vane of xxmaj don xxmaj bluth 's artwork would suit a better xxmaj

In [40]:
proc_num.deproc1(ll.train[0][0])[:10]

['xxbos',
 'xxmaj',
 'some',
 'wonder',
 'why',
 'there',
 'were',
 "n't",
 'anymore',
 'xxmaj']

## Batching

In [41]:
from IPython.display import display, HTML
import pandas as pd

Example:

In [42]:
text

"Some wonder why there weren't anymore Mrs. Murphy movies after this one. Will it's because this movie totally blew snot. Disney was not the right studio to run this film. MAYBE Touchstone (well, they're owned by Disney, but it'd be more adult). The film is too kid-ish, as the book series is not. The casting is all wrong for the characters. The characters don't even act the way they do in the books. And why was Tucker changed to a guy? He's a girl in the frigging books! Was this done to make the film appeal to boys? Sheesh. And where was Pewter, the gray cat? One of the funniest characters from the book is absent from this filth. Rita Mae Brown is a good writer, but letting Disney blow her work was wrong. An animated feature film, perhaps in the vane of Don Bluth's artwork would suit a better Mrs. Murphy film. Overall, I give this a 2, because at least Disney made a film from an under-appreciated book series. But, I wish they did better. Either way, I still have my books to entertain m

In [43]:
tokens = np.array(tp([text])[0])

In [44]:
tokens

array(['xxbos', 'xxmaj', 'some', 'wonder', ..., 'entertain', 'me', '.', 'xxeos'], dtype='<U11')

Let's say we split this into 6 batches of sequence length 15:

In [45]:
bs, seq_len = 6, 15

In [46]:
d_tokens = np.array([tokens[i * seq_len:(i+1)*seq_len] for i in range(bs)])

In [47]:
df = pd.DataFrame(d_tokens)

In [48]:
display(HTML(df.to_html(index=False, header=None)))

xxbos,xxmaj,some,wonder,why,there,were,n't,anymore,xxmaj,mrs.,xxmaj,murphy,movies,after
this,one,.,xxmaj,will,it,'s,because,this,movie,totally,blew,snot,.,xxmaj
disney,was,not,the,right,studio,to,run,this,film,.,xxup,maybe,xxmaj,touchstone
(,well,",",they,'re,owned,by,xxmaj,disney,",",but,it,'d,be,more
adult,),.,xxmaj,the,film,is,too,kid,-,ish,",",as,the,book
series,is,not,.,xxmaj,the,casting,is,all,wrong,for,the,characters,.,xxmaj


For a `bptt` of 5, we would go over the following three batches:

In [49]:
bs, bptt = 6, 5

In [50]:
for k in range(3):
    d_tokens = np.array([tokens[i*seq_len + k*bptt:i*seq_len + (k+1)*bptt] for i in range(bs)])
    df = pd.DataFrame(d_tokens)
    display(HTML(df.to_html(index=False, header=None)))

xxbos,xxmaj,some,wonder,why
this,one,.,xxmaj,will
disney,was,not,the,right
(,well,",",they,'re
adult,),.,xxmaj,the
series,is,not,.,xxmaj


there,were,n't,anymore,xxmaj
it,'s,because,this,movie
studio,to,run,this,film
owned,by,xxmaj,disney,","
film,is,too,kid,-
the,casting,is,all,wrong


mrs.,xxmaj,murphy,movies,after
totally,blew,snot,.,xxmaj
.,xxup,maybe,xxmaj,touchstone
but,it,'d,be,more
ish,",",as,the,book
for,the,characters,.,xxmaj


In [51]:
#export
class LM_PreLoader():
    def __init__(self, data, bs=64, bptt=70, shuffle=False):
        self.data, self.bs, self.bptt, self.shuffle = data, bs, bptt, shuffle
        total_len = sum([len(t) for t in data.x])
        self.n_batch = total_len // bs  # len of each of the bs concatenated streams
        self.batchify()
        
    def __len__(self):
        return ((self.n_batch - 1) // self.bptt) * self.bs  # How many token sequences of len bptt and "bs=1"
    
    def __getitem__(self, idx):
        source = self.batched_data[idx % self.bs]  # which row in the (bs, n_batch) tensor
        seq_idx = (idx // self.bs) * self.bptt     # idx of the substring of len bptt in source/one row
        return source[seq_idx:seq_idx+self.bptt],source[seq_idx+1:seq_idx+self.bptt+1]
    
    def batchify(self):
        texts = self.data.x
        if self.shuffle: texts = texts[torch.randperm(len(texts))]
        
        stream = torch.cat([tensor(t) for t in texts])
        
        self.batched_data = stream[:self.n_batch * self.bs].view(self.bs, self.n_batch)

*Helpful experiments to understand:*

In [52]:
def visualize_tensor(t):
    display(HTML(pd.DataFrame(np.array(t)).to_html(index=False, header=None)))

In [53]:
stream = torch.cat([tensor(t) for t in ll.train.x])

In [54]:
bs = 3
bptt = 5
n_batch = 15

In [55]:
batched_data = stream[:bs * n_batch].view(bs,n_batch); batched_data.shape

torch.Size([3, 15])

In [56]:
visualize_tensor(batched_data)

2,7,65,606,154,54,86,35,1561,7,2293,7,2128,118,119
19,43,9,7,105,16,22,107,19,29,480,4472,15067,9,7
998,25,37,8,227,1154,14,502,19,31,9,6,298,7,32086


In [57]:
idx = 4

In [58]:
source = batched_data[idx % bs]

In [59]:
visualize_tensor(source.unsqueeze(0))

19,43,9,7,105,16,22,107,19,29,480,4472,15067,9,7


In [60]:
seq_idx = (idx // bs) * bptt; seq_idx

5

In [61]:
visualize_tensor(source[seq_idx:seq_idx + bptt].unsqueeze(0))
visualize_tensor(source[seq_idx+1:seq_idx+bptt+1].unsqueeze(0))

16,22,107,19,29


22,107,19,29,480


In [62]:
lm_pre = LM_PreLoader(ll.valid, shuffle=True)

In [63]:
len(lm_pre)

43392

In [64]:
sum([len(t) for t in ll.valid.x])//70

43443

In [65]:
x, y = lm_pre[10]

In [66]:
visualize_tensor(x.unsqueeze(0))
visualize_tensor(y.unsqueeze(0))

1139,113,16,28,112,9,7,1222,12,833,10,225,126,10,1139,9,7,17,8,148,7,6838,3418,5581,8,440,14,1287,9,24,7,1139,753,95,7,1567,7,1062,7,1351,7,4450,36,2938,33,3111,7,10567,28,8,6,11465,125,2958,7,6193,23,7,100,302,1632,9,7,83,43,13,7,33396,22,139


113,16,28,112,9,7,1222,12,833,10,225,126,10,1139,9,7,17,8,148,7,6838,3418,5581,8,440,14,1287,9,24,7,1139,753,95,7,1567,7,1062,7,1351,7,4450,36,2938,33,3111,7,10567,28,8,6,11465,125,2958,7,6193,23,7,100,302,1632,9,7,83,43,13,7,33396,22,139,4804


In [67]:
dl = DataLoader(lm_pre, batch_size=64)

In [68]:
iter_dl = iter(dl)

In [69]:
x1, y1 = next(iter_dl)
x2, y2 = next(iter_dl)

In [70]:
x1.size(), y1.size()

(torch.Size([64, 70]), torch.Size([64, 70]))

In [71]:
vocab = proc_num.vocab

In [72]:
' '.join([vocab[o] for o in x1[0]])

'xxbos xxmaj great surprise from xxmaj peter xxmaj jackson on the xxmaj fellowship of the xxmaj ring xxmaj extended xxup dvd . xxmaj jack xxmaj black hands over the xxmaj one xxmaj ring at the xxmaj council of xxmaj elrond . xxmaj problem ? xxmaj he got drunk with some mates the night before and got is ... put somewhere personal . \n\n xxmaj spliced together with the scene from'

In [73]:
' '.join([vocab[o] for o in y1[0]])

'xxmaj great surprise from xxmaj peter xxmaj jackson on the xxmaj fellowship of the xxmaj ring xxmaj extended xxup dvd . xxmaj jack xxmaj black hands over the xxmaj one xxmaj ring at the xxmaj council of xxmaj elrond . xxmaj problem ? xxmaj he got drunk with some mates the night before and got is ... put somewhere personal . \n\n xxmaj spliced together with the scene from the'

In [74]:
' '.join([vocab[o] for o in x2[0]])

'the film , this is a great little treat for people who hate -- or love -- xxmaj the xxmaj lord of the xxmaj rings . xxeos xxbos xxmaj this is one of the funniest and most warm - hearted films ever ! xxmaj john xxmaj gordon xxmaj sinclair and xxmaj dee xxmaj hepburn were absolutely wonderful in this story of teenage love and the sudden twists & turns that'

In [75]:
' '.join([vocab[o] for o in y2[0]])

'film , this is a great little treat for people who hate -- or love -- xxmaj the xxmaj lord of the xxmaj rings . xxeos xxbos xxmaj this is one of the funniest and most warm - hearted films ever ! xxmaj john xxmaj gordon xxmaj sinclair and xxmaj dee xxmaj hepburn were absolutely wonderful in this story of teenage love and the sudden twists & turns that occur'

*Notice, that the target y has an offset of one to the right with respect to x!*

In [76]:
#export
def get_lm_dls(train_ds, valid_ds, bs, bptt, **kwargs):
    return (
        DataLoader(LM_PreLoader(train_ds, bs, bptt, shuffle=True), batch_size=bs, **kwargs),
        DataLoader(LM_PreLoader(valid_ds, bs, bptt, shuffle=False), batch_size=2*bs, **kwargs)
    )

In [77]:
#export
def lm_databunchify(sd, bs, bptt, **kwargs):
    return DataBunch(*get_lm_dls(sd.train, sd.valid, bs, bptt, **kwargs))

In [78]:
bs, bptt = 64, 70

In [79]:
data = lm_databunchify(ll, bs, bptt)

In [80]:
data.train_ds

In [81]:
len(data.train_ds)

387328

In [82]:
len(data.train_dl)

6052

In [83]:
6054 * bs

387456

In [84]:
next(iter(data.train_dl))[0].shape

torch.Size([64, 70])

## Batching for classification

In [85]:
proc_cat = CategoryProcessor()

In [86]:
il = TextList.from_files(path, include=['train', 'test'])

In [87]:
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name="test"))

In [88]:
ll = label_by_func(sd, parent_labeler, proc_x = [proc_tok, proc_num], proc_y=proc_cat)

In [89]:
pickle.dump(ll, open(path/"ll_clas.pkl", "wb"))

In [90]:
ll = pickle.load(open(path/"ll_clas.pkl", "rb"))

In [91]:
[(ll.train.x_obj(i), ll.train.y_obj(i)) for i in [1, 2355]];

For memory reasons it's good to have the biggest tensors first. Therefore we sort them this way for our `SortSampler`:

In [92]:
#export
from torch.utils.data import Sampler

class SortSampler(Sampler):
    def __init__(self, data_source, key):
        self.data_source, self.key = data_source, key
    def __len__(self):
        return len(self.data_source)
    def __iter__(self):
        return iter(sorted(list(range(len(self.data_source))), key=self.key, reverse=True))

For `SortishSampler`:

* Shuffle the text
* Build megabatches of `50 * bs`
* Sort these megabatches by length
* Split them into 50 minibatches

Gives randomized batches of roughly the same length.

In [93]:
#export
class SortishSampler(Sampler):
    def __init__(self, data_source, key, bs):
        self.data_source, self.key, self.bs = data_source, key, bs
        
    def __len__(self) -> int: return len(self.data_source)
    
    def __iter__(self):
        idxs = torch.randperm(len(self.data_source))
        megabatches = [idxs[i:i+self.bs*50] for i in range(0, len(idxs), self.bs*50)]  # list of tensors of ints
        sorted_idx = torch.cat([tensor(sorted(s, key=self.key, reverse=True)) for s in megabatches])
        batches = [sorted_idx[i:i+self.bs] for i in range(0, len(sorted_idx), self.bs)]  # list of tensors of ints
        
        max_idx = torch.argmax(tensor([self.key(ck[0]) for ck in batches]))  # find the chunk with the largest key, 0th element is largest
        
        # then make sure the largest sequence is first
        batches[0], batches[max_idx] = batches[max_idx], batches[0]
        
        batch_idxs = torch.randperm(len(batches) - 2)
        sorted_idx = torch.cat([batches[i+1] for i in batch_idxs]) if len(batches) > 1 else LongTensor([])
        sorted_idx = torch.cat([batches[0], sorted_idx, batches[-1]])
        return iter(sorted_idx)

In [94]:
#export
def pad_collate(samples, pad_idx=1, pad_first=False):
    max_len = max([len(s[0]) for s in samples])
    res = torch.zeros(len(samples), max_len).long() + pad_idx
    for i,s in enumerate(samples):
        if pad_first: res[i, -len(s[0]):] = LongTensor(s[0])
        else:         res[i, :len(s[0]) ] = LongTensor(s[0])
    return res, tensor([s[1] for s in samples])

In [95]:
bs = 64

In [96]:
train_sampler = SortishSampler(ll.train.x, key=lambda t: len(ll.train[int(t)][0]), bs=bs)

In [97]:
train_dl = DataLoader(ll.train, batch_size=bs, sampler=train_sampler, collate_fn=pad_collate)

In [98]:
iter_dl = iter(train_dl)

In [99]:
x, y = next(iter_dl)

In [100]:
lengths = []
for i in range(x.size(0)): lengths.append(x.size(1) - (x[i] == 1).sum().item())

In [101]:
lengths[:5], lengths[-1]

([3352, 1788, 1632, 1588, 1423], 990)

This was the first batch that contains the longest sequence. In the next batches the lengths are more similar:

In [102]:
x, y = next(iter_dl)
lengths = []
for i in range(x.size(0)): lengths.append(x.size(1) - (x[i] == 1).sum().item())
lengths[:5], lengths[-1]

([326, 326, 326, 326, 326], 315)

In [103]:
visualize_tensor(x)

2,7,301,714,12,75,65,477,101,113,12,29,59,12,230,20,208,338,104,759,10,1508,14,159,14,7,16344,10,30,83,81,14,65,1843,914,13,10954,17685,10,11,115,39,240,12,3466,8546,10,11,177,32,140,16,10,39,22,8308,17,12,1513,31379,430,14,2545,10,11,115,48,7377,2855,27,48,1901,23,3626,6443,292,17,11,240,4617,47,8,230,10,654,461,14,42,7166,44,143,8,287,30,132,209,14,1636,120,3079,10,11,115,6,9875,10,55,45,242,43,230,45,6,9875,10,240,3224,59,16,11,1832,43,102,230,17,14,2386,108,198,10,30,8,230,46,1458,14,2386,8,7166,230,60,14,159,363,11,41,6936,106,10,11,40,3071,23,6464,334,2668,8,18167,10,52,39,197,35,41,18167,10,11,115,39,60,14,159,61,11,185,28,8,7166,230,27,12,31525,4744,10,11,20,91,35,83,179,10,52,39,83,81,1119,8,4107,13,331,23,4617,4921,10,11,115,54,22,158,59,12,1999,10,11,127,1517,176,101,17,12,14010,23025,10,11,12,366,27,12,3361,15859,10,11,65,265,13,6522,4071,27,7488,23,667,2520,4910,10,11,252,18,197,35,404,32,109,8,118,657,10,44,18,78,157,15,7,3321,7,12526,15,17,16,1262,27,65,230,27,8,395,411,13,7,11720,45946,10,11,63,54,22,120,1473,17,8,195,308,3073,27,19,29,1094,12,4558,10,1073,343,11,25,2083,14,114,69,118,10,11,8,195,1402,6,3127,6,144,6,119,23,6,8,6,148,50,3
2,7,5260,44,18,258,12,6029,11840,252,8,603,13,59,892,9,18,461,18,258,205,2470,47,6029,10,30,2235,37,26,94,26,8,1373,122,17,19,31,9,24,7,17,19,31,10,8,1373,122,5643,12,164,6029,1784,462,7,16071,3170,7,0,49,2410,1681,17,20766,9,7,39,15,836,28,12,475,39,275,466,7,0,10,135,12,337,2162,25,275,47,7,0,9,7,318,13,43682,10,39,346,2384,61,13,8,9347,2324,10,132,14,1018,9,24,7,8,1099,15770,13,8,1373,122,15,492,12,1785,13,65,13,8,69,627,7,0,32,559,98,17,6029,36188,9,7,6029,18,1365,15,12,70,219,3596,10,11,32,78,559,148,72,49612,17,102,3826,13,136,9,12,688,512,13,19,15,8,6029,1850,7,3428,7,16867,9,7,284,12,1194,10,39,25,103,70,4135,17,129,757,9,24,7,17,8,31,10,12,7,195,6193,1139,15,2407,10,26,223,48,7,1046,7,28184,11,7,24371,9,7,8,2847,15,12,12135,10,79,178,8,195,7,32928,677,13,7,18201,60,559,5776,26,12,2847,413,23,182,51,38689,1788,14,56,263,17,8,486,9,7,30,19,25,12,541,12135,23,79,103,4367,12,828,2847,10,135,8,8660,13,273,541,319,466,273,102,9,24,7,8,2757,177,0,181,28,12,1456,6029,4737,17,413,9,7,17,212,10,18,41,244,12,6881,397,59,16,10,28,32,14,3838,8,1808,2162,17,1541,23,388,50,5639,95,125,125,0,125,38648,3249,0,7,139,3523,7,0,3
2,7,106,13,44,10,18,258,37,12,686,355,13,2457,7,4529,458,10,79,15,107,10,8,675,6277,13,1874,12,881,1034,1009,15,47,3358,1827,8,54269,9,7,19,29,15,180,13,48,1767,959,20,2087,27,15460,10,26,8,170,11,4531,7155,9467,9,7,17,77,2788,682,10,8,3903,1281,13,19,29,15,114,306,47,1371,12,2541,0,212,16,15,670,1271,7456,17,8,2074,659,33,7,697,38727,253,1631,27,12,1368,5213,13,64,18,1017,9,24,7,214,10,8,137,15,749,428,9,7,17,212,10,110,13,8,196,60,2299,946,11,783,267,14,7840,28,64,7,49688,3615,134,644,50,7,44,1078,13,8,196,86,926,1134,17,80,589,11,88,10865,28,8,320,10,76,8,360,2339,703,9,36,7,53,7,0,33,7,45,242,10,7,49688,1029,12,418,678,13,4818,10,56,107,39,696,88,109,14,2196,8,196,9,7,102,93,20,10,39,15,56,12,1926,3294,10,49,25702,332,12,170,27,48,1767,644,9,7,233,34,10,553,15,37,158,20,4240,3193,13,1980,27,1260,125,331,1260,366,9,7,11,56,107,1926,6103,142,10,74,170,493,12,8257,17,8,497,13,7,4529,458,9,7,56,516,147,4221,11,1422,158,164,1767,10,18,258,1064,7961,14,8377,147,385,179,9,7,443,10,19,75,32,1954,14,2715,48,1767,1573,9,24,7,17,364,134,7,7312,4024,7,67,137,10,901,366,36,56,3971,50,33,3215,7,6924,4024,7,273,11,191,694,169,10,102,93,8,196,3
2,7,166,10,18,41,35,371,8,232,364,82,14,140,44,8,5182,798,20,436,383,150,10,52,18,167,37,182,14,159,198,20,2641,150,9,24,7,30,18,41,65,75,624,12869,20,7,1892,7,748,118,346,0,32427,201,124,10,27,401,12,381,13,5277,9,7,19,25,37,43,13,112,9,7,16,679,88,217,10,11,28,301,18,167,37,182,14,3905,59,8,137,10,284,7,2115,7,17061,25,26,675,421,400,7,103,18,105,3316,8,938,621,13,15594,1915,10,26,46,86,1708,14,114,8,82,150,831,944,92,24,7,30,54,25,43,4591,243,20,18,96,35,367,30,98,3685,27,9,7,11,20,410,206,516,14,8,148,13,8,29,10,11,45,242,263,121,4

Add convenience functions:

In [104]:
#export
def get_clas_dls(train_ds, valid_ds, bs, **kwargs):
    train_sampler = SortishSampler(train_ds.x, key=lambda t: len(train_ds.x[t]), bs=bs)
    
    valid_sampler = SortSampler(valid_ds.x, key=lambda t: len(valid_ds.x[t]))
    
    return (
        DataLoader(train_ds, batch_size=bs, sampler=train_sampler, collate_fn=pad_collate, **kwargs),
        DataLoader(valid_ds, batch_size=2*bs, sampler=valid_sampler, collate_fn=pad_collate, **kwargs)
    )

In [105]:
#export
def clas_databunchify(sd, bs, **kwargs):
    return DataBunch(*get_clas_dls(sd.train, sd.valid, bs, **kwargs))

In [106]:
bs, bptt = 64, 70

In [107]:
data = clas_databunchify(ll, bs)

In [108]:
x, y = next(iter(data.train_dl))

In [109]:
x.shape, y.shape

(torch.Size([64, 3352]), torch.Size([64]))

In [110]:
x, y = next(iter(data.valid_dl))

In [111]:
x.shape, y.shape

(torch.Size([128, 2901]), torch.Size([128]))

## Export


In [113]:
!python notebook2script.py 12_text.ipynb

Converted 12_text.ipynb to exp/nb_12.py
